<a href="https://colab.research.google.com/github/VijiKK/Parallel_Algorithms/blob/main/mpi_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MPI demo 1**

# Before executing MPI code, **change runtime type to GPU**

In [1]:
!nvidia-smi

Mon Aug 28 23:16:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%writefile mpi_demo_1.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char *argv[])
{
    int rank, np;

    MPI_Init(&argc, &argv);

    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &np);

    //if (np != 2)
    //{
     //   MPI_Abort(MPI_COMM_WORLD, -1);
    //}


    int msg=0;

    if (rank == 0)
    {
        msg=9995823;

        // send 1 int to rank=1
	for(int i=1; i<5; i++){
        MPI_Send(&msg, 1, MPI_INT, i, 0, MPI_COMM_WORLD);
        printf ("%d -- sent message\n", rank);
  	}

    }
    else                        /* rank must be 1 since np=2 */
    {
        MPI_Recv(&msg, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf ("%d -- received message %d\n", rank, msg);
    }



    MPI_Finalize();
    return 0;
}


Writing mpi_demo_1.c


In [9]:
!mpicc mpi_demo_1.c -o mpi_demo_1_out.out && mpirun -n 4 --use-hwthread-cpus --oversubscribe --allow-run-as-root mpi_demo_1_out.out


0 -- sent message
1 -- received message 9995823
[6946e2f0e585:02127] *** An error occurred in MPI_Send
[6946e2f0e585:02127] *** reported by process [1481900033,0]
[6946e2f0e585:02127] *** on communicator MPI_COMM_WORLD
[6946e2f0e585:02127] *** MPI_ERR_RANK: invalid rank
[6946e2f0e585:02127] *** MPI_ERRORS_ARE_FATAL (processes in this communicator will now abort,
[6946e2f0e585:02127] ***    and potentially your MPI job)


# **MPI demo 2**

In [6]:
%%writefile mpi_demo_2.c

#include <mpi.h>
#include <stdio.h>

int main(int argc, char *argv[])
{
    int rank, np;

    MPI_Init(&argc, &argv);

    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &np);

    if (np != 2)
    {
        MPI_Abort(MPI_COMM_WORLD, -1);
    }


    if (rank == 0)
    {
        int send_msg=9995823;
        int recv_msg=0;

        MPI_Request request[2];


        // send 1 int to rank=1
        MPI_Isend(&send_msg, 1, MPI_INT, 1, 0, MPI_COMM_WORLD,
                  &request[0]);
        printf ("%d -- sent message\n", rank);


        MPI_Irecv(&recv_msg, 1, MPI_INT, 1, 0, MPI_COMM_WORLD,
                  &request[1]);
        printf ("%d -- received message %d\n", rank, recv_msg);


        // calculate??

        MPI_Wait(&request[0], MPI_STATUS_IGNORE);
        MPI_Wait(&request[1], MPI_STATUS_IGNORE);
        // now it's safe to access/modify message data
        printf ("%d -- received message %d\n", rank, recv_msg);

    }
    else                        /* rank must be 1 since np=2 */
    {
        int send_msg=1111111;
        int recv_msg=0;

        MPI_Request request[2];

        MPI_Isend(&send_msg, 1, MPI_INT, 0, 0, MPI_COMM_WORLD,
                  &request[0]);
        printf ("%d -- sent message\n", rank);

        MPI_Irecv(&recv_msg, 1, MPI_INT, 0, 0, MPI_COMM_WORLD,
                  &request[1]);
        printf ("%d -- received message %d\n", rank, recv_msg);

        MPI_Wait(&request[0], MPI_STATUS_IGNORE);
        MPI_Wait(&request[1], MPI_STATUS_IGNORE);
        printf ("%d -- received message %d\n", rank, recv_msg);


    }



    MPI_Finalize();
    return 0;
}

Writing mpi_demo_2.c


In [8]:
!mpicc mpi_demo_2.c -o mpi_demo_2_out.out && mpirun -n 2 --use-hwthread-cpus --oversubscribe --allow-run-as-root mpi_demo_2_out.out


1 -- sent message
1 -- received message 0
1 -- received message 9995823
0 -- sent message
0 -- received message 0
0 -- received message 1111111
